<a href="https://colab.research.google.com/github/GawainGan/LLM/blob/main/Module%202/LLM_Module_2_2_Transformer_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi # 用于检查现在使用的是CPU还是GPU， 如果是CPU，要转换成现有的GPU加速器，这里是T4

Fri Jul  5 22:56:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-05 23:15:07--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-05 23:15:07 (51.6 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# T5Tokenizer 是一个分词器类：
	# 标记化（Tokenization）：将输入文本分割成标记（tokens），即模型输入的基本单位。
	# 编码（Encoding）：将标记转换为对应的数字 ID，模型只能处理这些数字 ID。
	# 解码（Decoding）：将模型生成的数字 ID 转换回人类可读的文本。
  # 处理特殊标记 (Special Tokens)：如开始标记、结束标记、填充标记等，分词器负责在适当的位置插入这些标记。

# T5ForConditionalGeneration 是加载和推理 T5 模型的类：
  # T5 模型是一种基于 Transformer 架构的预训练模型，可以执行多种自然语言处理任务，包括翻译、摘要生成、问答等。
  # 条件生成 (Conditional Generation)：基于给定的输入生成相应的输出文本，如翻译、摘要生成等。
	# 推理 (Inference)：根据输入数据进行预测，输出结果通常是数字 ID，代表生成的标记。
	# 训练 (Training)：在训练过程中，模型根据损失函数调整参数，以提高任务的准确性。

分词器和模型在自然语言处理任务中扮演不同的角色，分别处理不同类型的任务。下面详细说明分词器（`T5Tokenizer`）和模型（`T5ForConditionalGeneration`）的区别：

### 分词器 (Tokenizer)

#### 主要功能
分词器的主要功能是将自然语言文本转换为模型可以理解的格式，并将模型输出的数字 ID 转换回可读文本。具体功能包括：
1. **标记化 (Tokenization)**：将输入文本拆分成词或子词（标记）。
2. **编码 (Encoding)**：将标记转换为数字 ID，模型只能处理这些数字 ID。
3. **解码 (Decoding)**：将模型生成的数字 ID 转换回人类可读的文本。
4. **处理特殊标记 (Special Tokens)**：如开始标记、结束标记、填充标记等，分词器负责在适当的位置插入这些标记。

#### 用法示例
```python
from transformers import T5Tokenizer

# 加载预训练的 T5 分词器
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")

# 输入文本
input_text = "Translate English to French: The house is wonderful."

# 标记化和编码
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 打印编码后的 ID
print(input_ids)

# 解码输出
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print(decoded_text)
```

### 模型 (Model)

#### 主要功能
模型的主要功能是基于输入的数字 ID（由分词器编码的结果）执行计算，以生成输出。具体功能包括：
1. **条件生成 (Conditional Generation)**：基于给定的输入生成相应的输出文本，如翻译、摘要生成等。
2. **推理 (Inference)**：根据输入数据进行预测，输出结果通常是数字 ID，代表生成的标记。
3. **训练 (Training)**：在训练过程中，模型根据损失函数调整参数，以提高任务的准确性。

#### 用法示例
```python
from transformers import T5ForConditionalGeneration

# 加载预训练的 T5 模型，并自动选择设备
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

# 使用分词器编码后的输入 ID 进行生成
outputs = model.generate(input_ids)

# 打印生成的输出 ID
print(outputs)

# 使用分词器解码生成的输出 ID 为可读文本
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)
```

### 总结

- **分词器 (`T5Tokenizer`)**
  - **任务**：将自然语言文本转换为模型可以处理的格式（标记和数字 ID），并将模型的输出转换回可读文本。
  - **作用**：预处理和后处理。
  - **示例**：`tokenizer.encode()`、`tokenizer.decode()`。

- **模型 (`T5ForConditionalGeneration`)**
  - **任务**：基于输入的数字 ID 执行计算并生成输出，处理实际的自然语言处理任务（如翻译、摘要生成）。
  - **作用**：核心计算和生成。
  - **示例**：`model.generate()`。

这两个组件一起使用，可以实现完整的自然语言处理任务，从文本输入到文本输出的整个流程。

In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
# "google/flan-t5-xl" 是预训练模型的名称
# 这个方法将下载分词器的配置和词汇表，并加载它们
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# 这个函数将构建的提示输入到预训练的 T5 模型中，生成回答。
  # prompt：由 build_prompt 函数生成的提示字符串。
  # generate_params：可选的生成参数字典，用于控制生成过程的行为。
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    # 将上面生成的prompt放入T5Tokenizer的分词器，
    # 将提示字符串分词并转换为模型输入格式
    # 流程：
      # 1.分词：分词器将自然语言文本（prompt）分割成标记，并将这些标记转换为对应的 ID。
	    # 2.转换为张量：分词器返回一个包含这些 ID 的张量。
	    # 3.移动到 GPU：将这个张量移动到 GPU，以利用 GPU 的加速计算能力。
    # 调用 tokenizer，return_tensors="pt" 参数指定分词器将输出转换为 PyTorch 张量（tensor）。这里的 pt 表示 PyTorch。
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    # 分词器返回一个包含多个张量的字典，input_ids 是其中的一个关键张量，表示输入文本的标记（tokens）的 ID。
	  # input_ids 是一个 PyTorch 张量，每个元素是一个整数，表示文本中一个标记的 ID。
    # .to("cuda")：将张量移动到 GPU

    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),    # 生成的标记（tokens）的最大数量
        num_beams=generate_params.get("num_beams", 5),        # 设置束搜索（beam search）中的束数，即同时跟踪多个候选序列，选择最优的序列
        do_sample=generate_params.get("do_sample", False),    # True：模型会基于概率进行采样，从而生成更具多样性的文本；False：则会选择概率最高的标记，生成的文本通常较为确定和重复
        temperature=generate_params.get("temperature", 1.0),  # (>1.0）会使得分布更均匀（增加随机性）;<1.0）会使得分布更集中（减少随机性）。通常用于调整生成的文本是更有创造性还是更确定
        top_k=generate_params.get("top_k", 50),               # 在每一步生成时，只考虑 top_k 个最可能的标记进行采样。这有助于减少不太可能的标记，提高生成文本的质量和连贯性
        top_p=generate_params.get("top_p", 0.95),             # 在每一步生成时，只考虑累计概率超过 top_p 的标记。这种方法通常可以生成更高质量和多样性的文本，因为它考虑了所有可能性较高的标记
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [10]:
llm("I just discovered the course. Can I still join in?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Yes, you are welcome to join us.'

In [11]:
llm("I failed to run docker. How to fix it?")

'Ensure that docker is up to date.'

In [12]:
rag("I just discovered the course. Can I still join in?")

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [13]:
search_result = search("I just discovered the course. Can I still join in?")
search_result[0]
# 很明显，我们的search_result返回了5条基于该问题的回复，当我们把这几个回复作为模版放入llm后，我们便是让llm本身选择一个最适合他的答案并且返回
# 如果我们修改了rag内部的参数，比如llm的max_length,temperature,generate_params.get, etc,那么得到的可能是不通的结果

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

### 对llm参数的理解

### 1. `max_length`
```python
max_length=generate_params.get("max_length", 200)
```
- **作用**：设置生成的文本的最大长度，即生成的标记（tokens）的最大数量。
- **默认值**：200
- **用途**：防止生成的文本过长，确保生成的内容在合理的范围内。

### 2. `num_beams`
```python
num_beams=generate_params.get("num_beams", 5)
```
- **作用**：设置束搜索（beam search）中的束数。
- **默认值**：5
- **用途**：束搜索是一种提高生成文本质量的技术，通过同时跟踪多个候选序列，选择最优的序列。更高的束数通常会产生更好的结果，但也会增加计算量。

### 3. `do_sample`
```python
do_sample=generate_params.get("do_sample", True)
```
- **作用**：是否进行采样生成。
- **默认值**：True
- **用途**：如果为 `True`，模型会基于概率进行采样，从而生成更具多样性的文本。如果为 `False`，则会选择概率最高的标记，生成的文本通常较为确定和重复。

### 4. `temperature`
```python
temperature=generate_params.get("temperature", 1.0)
```
- **作用**：控制生成文本的随机性。
- **默认值**：1.0
- **用途**：较高的温度（>1.0）会使得分布更均匀（增加随机性），较低的温度（<1.0）会使得分布更集中（减少随机性）。通常用于调整生成的文本是更有创造性还是更确定。

### 5. `top_k`
```python
top_k=generate_params.get("top_k", 50)
```
- **作用**：限制采样时考虑的标记数目。
- **默认值**：50
- **用途**：在每一步生成时，只考虑 `top_k` 个最可能的标记进行采样。这有助于减少不太可能的标记，提高生成文本的质量和连贯性。

### 6. `top_p`
```python
top_p=generate_params.get("top_p", 0.95)
```
- **作用**：设置核采样（nucleus sampling）的累积概率阈值。
- **默认值**：0.95
- **用途**：在每一步生成时，只考虑累计概率超过 `top_p` 的标记。这种方法通常可以生成更高质量和多样性的文本，因为它考虑了所有可能性较高的标记。

- **`max_length`**：限制生成文本的最大长度。
- **`num_beams`**：束搜索中的束数，提高生成质量。
- **`do_sample`**：控制是否进行采样，增加生成的多样性。
- **`temperature`**：调整生成的随机性和创造性。
- **`top_k`**：限制每步考虑的标记数，提升文本质量。
- **`top_p`**：基于累积概率进行采样，提升文本多样性和连贯性。

通过调整这些参数，你可以微调生成的文本，以满足特定需求和偏好。

In [15]:
def llm_2(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    # 将上面生成的prompt放入T5Tokenizer的分词器，
    # 将提示字符串分词并转换为模型输入格式
    # 流程：
      # 1.分词：分词器将自然语言文本（prompt）分割成标记，并将这些标记转换为对应的 ID。
	    # 2.转换为张量：分词器返回一个包含这些 ID 的张量。
	    # 3.移动到 GPU：将这个张量移动到 GPU，以利用 GPU 的加速计算能力。
    # 调用 tokenizer，return_tensors="pt" 参数指定分词器将输出转换为 PyTorch 张量（tensor）。这里的 pt 表示 PyTorch。
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    # 分词器返回一个包含多个张量的字典，input_ids 是其中的一个关键张量，表示输入文本的标记（tokens）的 ID。
	  # input_ids 是一个 PyTorch 张量，每个元素是一个整数，表示文本中一个标记的 ID。
    # .to("cuda")：将张量移动到 GPU

    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),    # 生成的标记（tokens）的最大数量
        num_beams=generate_params.get("num_beams", 5),        # 设置束搜索（beam search）中的束数，即同时跟踪多个候选序列，选择最优的序列
        do_sample=generate_params.get("do_sample", True),     # True：模型会基于概率进行采样，从而生成更具多样性的文本；False：则会选择概率最高的标记，生成的文本通常较为确定和重复
        temperature=generate_params.get("temperature", 0.9),  # (>1.0）会使得分布更均匀（增加随机性）;<1.0）会使得分布更集中（减少随机性）。通常用于调整生成的文本是更有创造性还是更确定
        top_k=generate_params.get("top_k", 90),               # 在每一步生成时，只考虑 top_k 个最可能的标记进行采样。这有助于减少不太可能的标记，提高生成文本的质量和连贯性
        top_p=generate_params.get("top_p", 0.95),             # 在每一步生成时，只考虑累计概率超过 top_p 的标记。这种方法通常可以生成更高质量和多样性的文本，因为它考虑了所有可能性较高的标记
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

def rag_2(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_2(prompt)
    return answer


In [16]:
llm_2("I failed to run docker")

''

In [16]:
rag_2("I failed to run docker?")

OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB. GPU 